# Superconducting Qubit Tune-up with LabOne Q for SHF Instruments and many qubits in parallel

In this notebook we demonstrate qubit tuneup with LabOne Q for many qubits in parallel, implemented as a sequence of experiments. 

Before starting the experiments, we define a set of initial qubit parameters, as might be known from fabrication. 

These parameters can then be used to update the baseline calibration used in the experiments and defined as part of the device setup.

## 0. General Imports and Definitions
### 0.1 Python Imports 

In [1]:
# LabOne Q:
import matplotlib.pyplot as plt
import numpy as np
from laboneq.contrib.example_helpers.generate_device_setup import (
    generate_device_setup_qubits,
)

# plotting functionality
from laboneq.contrib.example_helpers.plotting.plot_helpers import (
    plot_results,
)
from laboneq.simple import *

## 1. Device Setup

Below, you'll create a device setup and choose to run this notebook in emulated mode or directly on the control hardware, by specifying `use_emulation = True/False` respectively.

If you run on your hardware, you need to generate a device setup first, please have a look at our [device setup tutorial](https://docs.zhinst.com/labone_q_user_manual//tutorials/tutorials/01_device_setup/) for how to do this in general. 
Here, we use a helper functions to generate the device setup and a set up qubit objects with pre-defined parameters. 

In [ ]:
# specify the number of qubits you want to use
number_of_qubits = 6

# generate the device setup and the qubit objects using a helper function
device_setup, qubits = generate_device_setup_qubits(
    number_qubits=number_of_qubits,
    pqsc=[{"serial": "DEV10001"}],
    hdawg=[
        {
            "serial": "DEV8001",
            "number_of_channels": 8,
            "options": None,
        }
    ],
    shfqc=[
        {
            "serial": "DEV12001",
            "number_of_channels": 6,
            "readout_multiplex": 6,
            "options": None,
        }
    ],
    include_flux_lines=True,
    server_host="localhost",
    setup_name=f"my_{number_of_qubits}_tunable_qubit_setup",
)

### 2.2 Create and Connect to a LabOne Q Session 

Establishes the connection to the instruments and readies them for experiments

In [ ]:
# perform experiments in emulation mode only? - if True, also generate dummy data for fitting
emulate = True

# create and connect to a session
session = Session(device_setup=device_setup)
session.connect(do_emulation=emulate)

## 3. Qubit Tuneup - Experimental Sequence

Sequence of experiments for tuneup from scratch of a superconducting qubit in circuit QED architecture 

### 3.1 Resonator Spectroscopy: CW in parallel over full range of 0.5 - 8.5 GHz

Find the resonance frequency of the qubit readout resonator by looking at the transmission or reflection of a probe signal applied through the readout line

#### 3.1.1 Experiment Definition

Define the experimental pulse and readout sequence - here without any explicit qubit reference

Explicit qubit reference is then given through different experimental calibration and signal maps

In [4]:
# define sweep parameter
def create_freq_sweep(identifier, start_freq, stop_freq, num_points):
    return LinearSweepParameter(
        uid=f"frequency_sweep_{identifier}",
        start=start_freq,
        stop=stop_freq,
        count=num_points,
    )

In [5]:
def resonator_spectroscopy_parallel_cw(
    qubits, outer_sweeps, inner_sweeps, integration_time=10e-3, num_averages=1
):
    # Create resonator spectroscopy experiment - uses only readout drive and signal acquisition
    exp_spec = Experiment(
        uid="Resonator Spectroscopy",
        signals=[
            signal
            for signal_list in [
                [
                    ExperimentSignal(
                        f"measure_{qubit.uid}", map_to=qubit.signals["measure"]
                    ),
                    ExperimentSignal(
                        f"acquire_{qubit.uid}", map_to=qubit.signals["acquire"]
                    ),
                ]
                for qubit in qubits
            ]
            for signal in signal_list
        ],
    )

    ## define experimental sequence
    # loop - average multiple measurements for each frequency - measurement in spectroscopy mode
    with exp_spec.sweep(uid="resonator_frequency_outer", parameter=outer_sweeps):
        with exp_spec.acquire_loop_rt(
            uid="shots",
            count=num_averages,
            acquisition_type=AcquisitionType.SPECTROSCOPY,
        ):
            with exp_spec.sweep(
                uid="resonator_frequency_inner", parameter=inner_sweeps
            ):
                for qubit in qubits:
                    # readout pulse and data acquisition
                    with exp_spec.section(uid=f"resonator_spectroscopy_{qubit.uid}"):
                        # resonator signal readout
                        exp_spec.acquire(
                            signal=f"acquire_{qubit.uid}",
                            handle=f"resonator_spectroscopy_{qubit.uid}",
                            length=integration_time,
                        )
                    # relax time after readout - for qubit relaxation to groundstate and signal processing
                    with exp_spec.section(uid=f"relax_{qubit.uid}", length=1e-6):
                        exp_spec.reserve(signal=f"measure_{qubit.uid}")

    cal = Calibration()
    for it, qubit in enumerate(qubits):
        measure_acquire_oscillator = Oscillator(
            uid=f"{qubit.uid}_measure_acquire_oscillator", frequency=inner_sweeps[it]
        )
        measure_acquire_local_oscillator = Oscillator(frequency=outer_sweeps[it])
        cal[f"measure_{qubit.uid}"] = SignalCalibration(
            oscillator=measure_acquire_oscillator,
            local_oscillator=measure_acquire_local_oscillator,
            range=-10,
        )
        cal[f"acquire_{qubit.uid}"] = SignalCalibration(
            local_oscillator=measure_acquire_local_oscillator,
            oscillator=measure_acquire_oscillator,
            range=-5,
            port_delay=250e-9,
        )
    exp_spec.set_calibration(cal)

    return exp_spec

#### 3.1.2 Run and Evaluate Experiment
Runs the experiment and evaluates the data returned by the measurement

In [ ]:
# create experiment with outer, near-time sweep from 1-8 GHz in 1 GHz steps and a sweep over 1001 points within each 1GHz band - in parallel for all 4 QA channels
cw_spectroscopy_exp = resonator_spectroscopy_parallel_cw(
    qubits[::6],
    [create_freq_sweep(f"outer_{qubit.uid}", 1e9, 8e9, 8) for qubit in qubits[::6]],
    [
        create_freq_sweep(f"inner_{qubit.uid}", -500e6, 500e6, 1001)
        for qubit in qubits[::6]
    ],
)

compiled_cw_spectroscopy_exp = session.compile(cw_spectroscopy_exp)
cw_spectroscopy_results = session.run(compiled_cw_spectroscopy_exp)

In [ ]:
# access and plot results of one 8GHz sweep
full_data = cw_spectroscopy_results.get_data("resonator_spectroscopy_q0")

if number_of_qubits / 6 <= 1:
    outer = cw_spectroscopy_results.get_axis("resonator_spectroscopy_q0")[0]
    inner = cw_spectroscopy_results.get_axis("resonator_spectroscopy_q0")[1]
else:
    outer = cw_spectroscopy_results.get_axis("resonator_spectroscopy_q0")[0][0]
    inner = cw_spectroscopy_results.get_axis("resonator_spectroscopy_q0")[1][0]
full_sweep = np.array(
    [item for item_list in [out + inner for out in outer] for item in item_list]
)

plt.plot(
    full_sweep,
    np.array(
        [item for item_list in [data for data in full_data] for item in item_list]
    ),
)

In [8]:
# Do analysis of data here

#### 3.1.3 Update Calibration 

In [9]:
# update qubit parameters from analysis
# for qubit in my_qubits:
#     qubit.parameters.readout_resonator_frequency = my_new_frequency
#     device_setup.set_calibration(qubit.calibration())

### 3.2 Pulsed Qubit Spectroscopy: in parallel over 100MHz range for each qubit

Find the resonance frequency of the qubit by looking at the change in resonator transmission when sweeping the frequency of a qubit excitation pulse

#### 3.2.1 Experiment Definition

The frequency sweep of the drive line can now be done in real time (was: near time in older software releases)

In [10]:
def qubit_spectroscopy_pulse(qubit):
    return pulse_library.const(
        uid=f"spectroscopy_pulse_{qubit.uid}",
        length=qubit.parameters.user_defined["readout_length"],
        amplitude=0.8,
        # can_compress=True,
    )


def readout_pulse(qubit):
    return pulse_library.const(
        uid=f"readout_pulse_{qubit.uid}",
        length=qubit.parameters.user_defined["readout_length"],
        amplitude=qubit.parameters.user_defined["readout_amplitude"],
    )


def integration_kernel(qubit):
    return pulse_library.const(
        uid=f"integration_kernel_{qubit.uid}",
        length=qubit.parameters.user_defined["readout_length"],
        amplitude=1,
    )

In [11]:
# function that returns a qubit spectroscopy experiment- accepts frequency sweep range as parameter
def qubit_spectroscopy_parallel(
    qubits, qspec_range=100e6, qspec_num=1001, num_averages=2**10
):
    # Create qubit spectroscopy Experiment - uses qubit drive, readout drive and data acquisition lines
    exp_qspec = Experiment(
        uid="Qubit Spectroscopy",
        signals=[
            signal
            for signal_list in [
                [
                    ExperimentSignal(
                        f"drive_{qubit.uid}", map_to=qubit.signals["drive"]
                    ),
                    ExperimentSignal(
                        f"measure_{qubit.uid}", map_to=qubit.signals["measure"]
                    ),
                    ExperimentSignal(
                        f"acquire_{qubit.uid}", map_to=qubit.signals["acquire"]
                    ),
                ]
                for qubit in qubits
            ]
            for signal in signal_list
        ],
    )

    # List of frequency sweeps for all qubits
    qubit_frequency_sweeps = [
        LinearSweepParameter(
            uid=f"{qubit.uid}_spectroscopy_sweep",
            start=qubit.parameters.drive_frequency_ge - qspec_range / 2,
            stop=qubit.parameters.drive_frequency_ge + qspec_range / 2,
            count=qspec_num,
        )
        for qubit in qubits
    ]

    # inner loop - real-time averaging - QA in integration mode
    with exp_qspec.acquire_loop_rt(
        uid="freq_shots",
        count=num_averages,
        acquisition_type=AcquisitionType.INTEGRATION,
    ):
        with exp_qspec.sweep(
            uid="qubit_frequency_sweep", parameter=qubit_frequency_sweeps
        ):
            for qubit in qubits:
                # qubit drive
                with exp_qspec.section(uid=f"{qubit.uid}_excitation"):
                    exp_qspec.play(
                        signal=f"drive_{qubit.uid}",
                        pulse=qubit_spectroscopy_pulse(qubit),
                    )
                # measurement
                with exp_qspec.section(
                    uid=f"readout_{qubit.uid}", play_after=f"{qubit.uid}_excitation"
                ):
                    exp_qspec.measure(
                        measure_signal=f"measure_{qubit.uid}",
                        measure_pulse=readout_pulse(qubit),
                        handle=f"{qubit.uid}_spectroscopy",
                        acquire_signal=f"acquire_{qubit.uid}",
                        integration_kernel=integration_kernel(qubit),
                        reset_delay=qubit.parameters.user_defined["reset_delay_length"],
                    )

    cal = Calibration()
    for it, qubit in enumerate(qubits):
        cal[f"drive_{qubit.uid}"] = SignalCalibration(
            oscillator=Oscillator(
                frequency=qubit_frequency_sweeps[it],
                modulation_type=ModulationType.HARDWARE,
            )
        )
    exp_qspec.set_calibration(cal)

    return exp_qspec

#### 3.2.2 Run and Evaluate Experiment for all Qubits in parallel

Runs the experiment and evaluates the data returned by the measurement

In [ ]:
qubit_spectroscopy_exp = qubit_spectroscopy_parallel(qubits)

compiled_qubit_spectroscopy_exp = session.compile(qubit_spectroscopy_exp)
qubit_spectroscopy_results = session.run(compiled_qubit_spectroscopy_exp)

In [ ]:
# access and plot results of one drive frequency sweep
index = 0
data_qubit = qubits[index]

qubit_data = qubit_spectroscopy_results.get_data(f"{data_qubit.uid}_spectroscopy")
qubit_freq = (
    qubit_spectroscopy_results.get_axis(f"{data_qubit.uid}_spectroscopy")[0][0]
    + data_qubit.parameters.drive_lo_frequency
)

plt.plot(qubit_freq, qubit_data)

In [14]:
# Do analysis of data here

#### 3.2.3 Update Calibration 

In [15]:
# update qubit parameters from analysis - here: qubit resonance frequency
# for qubit in my_qubits:
#     qubit.parameters.resonance_frequency_ge = my_new_frequency
#     device_setup.set_calibration(qubit.calibration())

### 3.3 Amplitude Rabi Experiment - in parallel

Sweep the pulse amplitude of a qubit drive pulse to determine the ideal amplitudes for specific qubit rotation angles

#### 3.3.1 Experiment Definition


In [16]:
def drive_ge_rabi(qubit):
    return pulse_library.drag(
        uid=f"drag_pulse_{qubit.uid}",
        length=qubit.parameters.user_defined["pulse_length"],
        sigma=0.4,
        beta=0.2,
        amplitude=1,
    )

In [17]:
# function that returns an amplitude Rabi experiment
def amplitude_rabi_parallel(qubits, amplitude_sweep, num_averages=2**10):
    exp_rabi = Experiment(
        uid="Qubit Spectroscopy",
        signals=[
            signal
            for signal_list in [
                [
                    ExperimentSignal(
                        f"drive_{qubit.uid}", map_to=qubit.signals["drive"]
                    ),
                    ExperimentSignal(
                        f"measure_{qubit.uid}", map_to=qubit.signals["measure"]
                    ),
                    ExperimentSignal(
                        f"acquire_{qubit.uid}", map_to=qubit.signals["acquire"]
                    ),
                ]
                for qubit in qubits
            ]
            for signal in signal_list
        ],
    )

    ## define Rabi experiment pulse sequence
    # outer loop - real-time, cyclic averaging
    with exp_rabi.acquire_loop_rt(
        uid="rabi_shots",
        count=num_averages,
        averaging_mode=AveragingMode.CYCLIC,
        acquisition_type=AcquisitionType.INTEGRATION,
    ):
        # inner loop - real time sweep of Rabi ampitudes
        with exp_rabi.sweep(uid="rabi_sweep", parameter=amplitude_sweep):
            for qubit in qubits:
                # qubit drive
                with exp_rabi.section(
                    uid=f"{qubit.uid}_excitation", alignment=SectionAlignment.RIGHT
                ):
                    exp_rabi.play(
                        signal=f"drive_{qubit.uid}",
                        pulse=drive_ge_rabi(qubit),
                        amplitude=amplitude_sweep,
                    )
                # measurement
                with exp_rabi.section(
                    uid=f"readout_{qubit.uid}", play_after=f"{qubit.uid}_excitation"
                ):
                    exp_rabi.measure(
                        measure_signal=f"measure_{qubit.uid}",
                        measure_pulse=readout_pulse(qubit),
                        handle=f"{qubit.uid}_rabi",
                        acquire_signal=f"acquire_{qubit.uid}",
                        integration_kernel=integration_kernel(qubit),
                        reset_delay=qubit.parameters.user_defined["reset_delay_length"],
                    )

    return exp_rabi

#### 3.3.2 Execute experiment and analyze results

In [ ]:
rabi_exp = amplitude_rabi_parallel(
    qubits,
    LinearSweepParameter(uid="amplitude_sweep", start=0, stop=1, count=201),
)

compiled_rabi_exp = session.compile(rabi_exp)
rabi_results = session.run(compiled_rabi_exp)

In [ ]:
# access and plot results of one drive frequency sweep
index = 0
data_qubit = qubits[index]

qubit_data = rabi_results.get_data(f"{data_qubit.uid}_rabi")
qubit_amp = rabi_results.get_axis(f"{data_qubit.uid}_rabi")[0]

plt.plot(qubit_amp, qubit_data)

In [ ]:
# plot all results
plot_results(rabi_results)

In [21]:
# Do analysis of data here

#### 3.3.3 Update Calibration 

In [22]:
# update qubit parameters from analysis - here: qubit pulse amplitude
# for qubit in my_qubits:
#     qubit.parameters.user_defined["amplitude_pi"] = my_amplitude

### 3.4 Ramsey Experiment - in parallel
The Ramsey experiment is different from the experiments above as the length of the drive section changes. Using a right-aligned sweep section and the automatic repetition time makes sure that the experiment is run as efficiently as possible on the Zurich Instruments hardware.

#### 3.4.1 Experiment Definition

In [23]:
# define ramsey drive pulse - use calibration from Rabi experiment
def drive_ge_pi_half(qubit):
    return pulse_library.drag(
        uid=f"ramsey_drive_{qubit.uid}",
        length=qubit.parameters.user_defined["pulse_length"],
        sigma=0.4,
        beta=0.2,
        amplitude=qubit.parameters.user_defined["amplitude_pi"] / 2,
    )

In [24]:
# function that returns an amplitude Rabi experiment
def ramsey_parallel(qubits, delay_sweep, num_averages=2**10):
    exp_ramsey = Experiment(
        uid="Qubit Spectroscopy",
        signals=[
            signal
            for signal_list in [
                [
                    ExperimentSignal(
                        f"drive_{qubit.uid}", map_to=qubit.signals["drive"]
                    ),
                    ExperimentSignal(
                        f"measure_{qubit.uid}", map_to=qubit.signals["measure"]
                    ),
                    ExperimentSignal(
                        f"acquire_{qubit.uid}", map_to=qubit.signals["acquire"]
                    ),
                ]
                for qubit in qubits
            ]
            for signal in signal_list
        ],
    )

    ## define Ramsey experiment pulse sequence
    # outer loop - real-time, cyclic averaging
    with exp_ramsey.acquire_loop_rt(
        uid="ramsey_shots",
        count=num_averages,
        averaging_mode=AveragingMode.CYCLIC,
        acquisition_type=AcquisitionType.INTEGRATION,
        repetition_mode=RepetitionMode.AUTO,
    ):
        # inner loop - real time sweep of Ramsey time delays
        with exp_ramsey.sweep(
            uid="ramsey_sweep", parameter=delay_sweep, alignment=SectionAlignment.RIGHT
        ):
            for qubit in qubits:
                # play qubit excitation pulse - pulse amplitude is swept
                ramsey_pulse = drive_ge_pi_half(qubit)
                with exp_ramsey.section(
                    uid=f"{qubit.uid}_excitation", alignment=SectionAlignment.RIGHT
                ):
                    exp_ramsey.play(signal=f"drive_{qubit.uid}", pulse=ramsey_pulse)
                    exp_ramsey.delay(signal=f"drive_{qubit.uid}", time=delay_sweep)
                    exp_ramsey.play(signal=f"drive_{qubit.uid}", pulse=ramsey_pulse)
                # readout pulse and data acquisition
                # measurement
                with exp_ramsey.section(
                    uid=f"readout_{qubit.uid}", play_after=f"{qubit.uid}_excitation"
                ):
                    exp_ramsey.measure(
                        measure_signal=f"measure_{qubit.uid}",
                        measure_pulse=readout_pulse(qubit),
                        handle=f"{qubit.uid}_ramsey",
                        acquire_signal=f"acquire_{qubit.uid}",
                        integration_kernel=integration_kernel(qubit),
                        reset_delay=qubit.parameters.user_defined["reset_delay_length"],
                    )

    return exp_ramsey

#### 3.4.2 Execute experiment

In [ ]:
ramsey_exp = ramsey_parallel(
    qubits,
    LinearSweepParameter(
        uid="ramsey_delay_sweep",
        start=0,
        stop=15e-6,
        count=201,
    ),
)

compiled_ramsey_exp = session.compile(ramsey_exp)
ramsey_results = session.run(compiled_ramsey_exp)

In [ ]:
# access and plot results of one drive frequency sweep
index = 0
data_qubit = qubits[index]

qubit_data = ramsey_results.get_data(f"{data_qubit.uid}_ramsey")
qubit_delay = ramsey_results.get_axis(f"{data_qubit.uid}_ramsey")[0]

plt.plot(qubit_delay, qubit_data)

In [ ]:
# plot all results
plot_results(ramsey_results)

In [28]:
# Do analysis of data here

#### 3.4.3 Update Qubit parameters

In [29]:
# update qubit parameters from analysis - here: qubit dephasing time
# for qubit in my_qubits:
#     qubit.parameters.user_defined["t2_time"] = my_t2

### 3.5 T1 Experiment - in parallel


#### 3.5.1 Experiment Definition

In [30]:
# define drive pulse - use calibration from Rabi experiment
def drive_ge_pi(qubit):
    return pulse_library.drag(
        uid=f"drive_{qubit.uid}",
        length=qubit.parameters.user_defined["pulse_length"],
        sigma=0.4,
        beta=0.2,
        amplitude=qubit.parameters.user_defined["amplitude_pi"],
    )

In [31]:
# function that returns an amplitude Rabi experiment
def t1_parallel(qubits, delay_sweep, num_averages=2**10):
    exp_t1 = Experiment(
        uid="Qubit Spectroscopy",
        signals=[
            signal
            for signal_list in [
                [
                    ExperimentSignal(
                        f"drive_{qubit.uid}", map_to=qubit.signals["drive"]
                    ),
                    ExperimentSignal(
                        f"measure_{qubit.uid}", map_to=qubit.signals["measure"]
                    ),
                    ExperimentSignal(
                        f"acquire_{qubit.uid}", map_to=qubit.signals["acquire"]
                    ),
                ]
                for qubit in qubits
            ]
            for signal in signal_list
        ],
    )

    ## define Ramsey experiment pulse sequence
    # outer loop - real-time, cyclic averaging
    with exp_t1.acquire_loop_rt(
        uid="t1_shots",
        count=num_averages,
        averaging_mode=AveragingMode.CYCLIC,
        acquisition_type=AcquisitionType.INTEGRATION,
        repetition_mode=RepetitionMode.AUTO,
    ):
        # inner loop - real time sweep of Ramsey time delays
        with exp_t1.sweep(
            uid="t1_delay_sweep",
            parameter=delay_sweep,
            alignment=SectionAlignment.RIGHT,
        ):
            for qubit in qubits:
                # play qubit excitation pulse - pulse amplitude is swept
                with exp_t1.section(
                    uid=f"{qubit.uid}_excitation", alignment=SectionAlignment.RIGHT
                ):
                    exp_t1.play(signal=f"drive_{qubit.uid}", pulse=drive_ge_pi(qubit))
                    exp_t1.delay(signal=f"drive_{qubit.uid}", time=delay_sweep)
                # readout pulse and data acquisition
                # measurement
                with exp_t1.section(
                    uid=f"readout_{qubit.uid}", play_after=f"{qubit.uid}_excitation"
                ):
                    exp_t1.measure(
                        measure_signal=f"measure_{qubit.uid}",
                        measure_pulse=readout_pulse(qubit),
                        handle=f"{qubit.uid}_t1",
                        acquire_signal=f"acquire_{qubit.uid}",
                        integration_kernel=integration_kernel(qubit),
                        reset_delay=qubit.parameters.user_defined["reset_delay_length"],
                    )

    return exp_t1

#### 3.5.2 Execute experiment

In [ ]:
t1_exp = t1_parallel(
    qubits,
    LinearSweepParameter(uid="t1_delay_sweep", start=0, stop=50e-6, count=201),
)

compiled_t1_exp = session.compile(t1_exp)
t1_results = session.run(compiled_t1_exp)

In [ ]:
# access and plot results of one drive frequency sweep
index = 0
data_qubit = qubits[index]

qubit_data = t1_results.get_data(f"{data_qubit.uid}_t1")
qubit_delay = t1_results.get_axis(f"{data_qubit.uid}_t1")[0]

plt.plot(qubit_delay, qubit_data)

In [ ]:
# plot all results
plot_results(t1_results)

In [35]:
# Do analysis of data here

#### 3.5.3 Update Qubit parameters 

In [36]:
# update qubit parameters from analysis - here: qubit relaxation time
# for qubit in my_qubits:
#     qubit.parameters.user_defined["t1_time"] = my_t1